Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = @__DIR__
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_04 = "
data{
    int N;
    int N_actors;
    int pulled_left[N];
    int prosoc_left[N];
    int condition[N];
    int actor[N];
}
parameters{
    vector[N_actors] a;
    real bp;
    real bpC;
}
model{
    vector[N] p;
    bpC ~ normal( 0 , 10 );
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + (bp + bpC * condition[i]) * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_10_04",
model=m_10_04, output_format=:mcmcchains);

Input data for cmdstan

In [6]:
m_10_04_data = Dict("N" => size(df, 1), "N_actors" => length(unique(df[:actor])),
"actor" => df[:actor], "pulled_left" => df[:pulled_left],
"prosoc_left" => df[:prosoc_left], "condition" => df[:condition]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_04_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
      mean   sd  5.5% 94.5% n_eff Rhat
bp    0.84 0.26  0.43  1.26  2271    1
bpC  -0.13 0.29 -0.59  0.34  2949    1

a[1] -0.74 0.27 -1.16 -0.31  3310    1
a[2] 10.88 5.20  4.57 20.73  1634    1
a[3] -1.05 0.28 -1.52 -0.59  4206    1
a[4] -1.05 0.28 -1.50 -0.60  4133    1
a[5] -0.75 0.27 -1.18 -0.32  4049    1
a[6]  0.22 0.27 -0.22  0.65  3877    1
a[7]  1.81 0.39  1.22  2.48  3807    1
";


Inference for Stan model: m_10_04_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (1.5, 1.4, 1.5, 1.4) seconds, 5.8 seconds total
Sampling took (1.3, 1.3, 1.3, 1.4) seconds, 5.3 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%  N_Eff  N_Eff/s    R_hat
lp__             -261  4.9e-02     2.1   -266   -261   -259   1852      349  1.0e+00
accept_stat__    0.84  5.5e-03    0.21   0.37   0.93    1.0   1399      264  1.0e+00
stepsize__       0.37  2.8e-02   0.039   0.32   0.40   0.42    2.0     0.38  5.8e+13
treedepth__       3.3  1.3e-01    0.63    2.0    3.0    4.0     23      4.4  1.1e+00
n_leapfrog__       12  1.1e-01     6.3    7.0     15     23   3295      621  1.0e+00
divergent__     0.079  4.7e-03    0.27   0.00   0.00    1.0   3238      610  1.0e+00
energy__          266  7.5e-02     3.0    262    266    271   1597      301  1.0e+00
a[1]            -0.74  5.0e-03    0.27   -1.2  -0.

Update sections

In [8]:
chn2 = set_section(chn, Dict(
  :parameters => ["bp", "bpC"],
  :pooled => ["a.$i" for i in 1:7],
  :internals => ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__",
    "divergent__", "energy__"]
  )
)

Object of type Chains, with data of type 1000×16×4 Array{Float64,3}

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a.1, a.2, a.3, a.4, a.5, a.6, a.7
internals         = lp__, accept_stat__, stepsize__, treedepth__, n_leapfrog__, divergent__, energy__
parameters        = bp, bpC

Summary Statistics

│ Row │ parameters │ mean      │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64   │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼───────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ bp         │ 0.833453  │ 0.265344 │ 0.00419545 │ 0.0074115  │ 4000.0 │
│ 2   │ bpC        │ -0.123269 │ 0.304098 │ 0.00480821 │ 0.00746478 │ 4000.0 │

Describe parameter draws

In [9]:
describe(chn2)

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean      │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64   │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼───────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ bp         │ 0.833453  │ 0.265344 │ 0.00419545 │ 0.0074115  │ 4000.0 │
│ 2   │ bpC        │ -0.123269 │ 0.304098 │ 0.00480821 │ 0.00746478 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%      │ 25.0%     │ 50.0%     │ 75.0%     │ 97.5%    │
│     │ Symbol     │ Float64   │ Float64   │ Float64   │ Float64   │ Float64  │
├─────┼────────────┼───────────┼───────────┼───────────┼───────────┼──────────┤
│ 1   │ bp         │ 0.324267  │ 0.649746  │ 0.833275  │ 1.01026   │ 1.36137  │
│ 2   │ bpC        │ -0.725581 │ -0.325243 │ -0.120988 │ 0.0748142 │ 0.474432 │


Describe pooled parameter draws

In [10]:
describe(chn2, sections=[:pooled])

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean      │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64   │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼───────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ bp         │ 0.833453  │ 0.265344 │ 0.00419545 │ 0.0074115  │ 4000.0 │
│ 2   │ bpC        │ -0.123269 │ 0.304098 │ 0.00480821 │ 0.00746478 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%      │ 25.0%     │ 50.0%     │ 75.0%     │ 97.5%    │
│     │ Symbol     │ Float64   │ Float64   │ Float64   │ Float64   │ Float64  │
├─────┼────────────┼───────────┼───────────┼───────────┼───────────┼──────────┤
│ 1   │ bp         │ 0.324267  │ 0.649746  │ 0.833275  │ 1.01026   │ 1.36137  │
│ 2   │ bpC        │ -0.725581 │ -0.325243 │ -0.120988 │ 0.0748142 │ 0.474432 │


Make it a DataFrame

In [11]:
df = DataFrame(chn2, [:parameters, :pooled])

,a.1,a.2,a.3,a.4,a.5,a.6,a.7,bp,bpC
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.0468,8.07213,-1.3285,-0.968215,-0.622832,0.403894,1.14205,0.838573,-0.473409
2,-0.784575,8.61046,-0.823028,-0.776454,-0.842335,0.243867,1.64324,0.697878,0.299866
3,-0.353663,7.66793,-0.423418,-0.555728,-0.373674,0.185303,1.48228,0.0396729,0.222009
4,-0.523634,7.29394,-1.27964,-0.782069,-0.413622,0.447424,1.92392,0.668755,-0.116054
5,-0.687086,8.21284,-0.578259,-1.06027,-1.01189,-0.0814459,1.73903,0.800018,-0.0898825
6,-0.635837,3.96245,-0.630827,-1.09904,-0.939307,-0.276528,1.59156,0.974214,-0.117283
7,-0.531541,5.59591,-0.429468,-1.00768,-0.676277,0.0835493,1.62964,0.873363,-0.376451
8,-0.70768,10.1341,-1.42485,-1.04445,-0.584926,0.347632,1.55725,0.610506,0.0144794
9,-0.742143,4.49312,-0.661353,-0.485559,-0.983997,0.802211,1.9216,0.67374,-0.0419584


End of `10/m10.04s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*